## 1. Parsing gene bank files

### Read GenBank
There are different option to open a genbank file and parse it. One can access to almost every feature and printed in a nice way. It is also possible to save the output in a file. We are going to use as an input the GB file that get got from our blastp search.

In [ ]:
from Bio import GenBank
#Replace 'name-file' with the GB file that you got from the blastp run
outputblastp = 'name-file.txt'

We are going to extract the record ID and the organism name.

In [ ]:
#Save in file, 'a' is an option to append and creates file if not existing
f=open('output-file.txt', 'a')#Create new file
with open(outputblast) as handle:
    for record in GenBank.parse(handle):
        info = [record. organism, "\t", record.version, '\n']
        f.writelines(info)

f.close()

Once we have the list pf organims with the proteinID, we are going to use it to access entrez

## 2. Accesing entrez using BioPython

In [1]:
import matplotlib.pyplot as plt
import numpy as np

#Pandas for data manipulation
import pandas as pd

from Bio import Entrez
from Bio import SeqIO

Go in ubuntu shell and extract unique organism names

cat output_blastpGBfile.gb | grep -e 'ORGANISM' | cut -d 'M' -f2 > organism_blastp100.txt

### Access genID from proteinID

We are gonna use the list of proteinID and organims that we fetch from the gb file from the blastp output. We will use the proteinID number to match in between the protein database and the nuccore database. The nuccore database has the position in the genome to check in our refseq files (GCF)

In [23]:
fromOutput = pd.read_csv('output-file.txt', sep = '\t', header = None)
fromOutput.columns = ["Organism", 'proteinID']
fromOutput.head()

,Organism,proteinID
0,Myxococcus fulvus,WP_074958386.1
1,Brevibacillus agri,WP_025848602.1
2,Nostoc punctiforme NIES-2108,RCJ36599.1
3,Brevibacillus sp. CF112,EJL39661.1
4,Salinispora pacifica,WP_018812186.1


In [19]:
fromOutput.iloc[0:10,1]

0        NER75497.1
1        OQY56350.1
2        NER75085.1
3    WP_089128067.1
4    WP_131121152.1
5        TBR56966.1
6    WP_017308762.1
7    WP_096657262.1
8        NEP58315.1
9    WP_020930534.1
Name: proteinID, dtype: object

#### Get all at once and save

In [24]:
#All at once 
Entrez.email = "dgarcia@eng.au.dk"
for i in fromOutput.iloc[:,1]:
    filename = i.split('.')
    #print(filename[0])
    handle = Entrez.elink(dbfrom = "protein", db = "nuccore", id = i)
    try:
        record = Entrez.read(handle)
        #print(record)
        #Fetch nuccore id
        if len(record[0]["LinkSetDb"]) == 2:
            net_handle = Entrez.efetch(db = "nuccore", id = record[0]["LinkSetDb"][0]["Link"][0]["Id"], rettype = "gb", retmode = "text")
            out_handle = open(filename[0]+'.gb', "w")
            out_handle.write(net_handle.read())
            out_handle.close()
            net_handle.close()
            #print can be removed if wanted
            print("Save", i)
        handle.close()
    except:
        pass


('Save', 'WP_074958386.1')
('Save', 'WP_025848602.1')
('Save', 'RCJ36599.1')
('Save', 'EJL39661.1')
('Save', 'WP_018812186.1')
('Save', 'WP_145920409.1')
('Save', 'WP_027657832.1')
('Save', 'WP_029130478.1')
('Save', 'WP_029122349.1')
('Save', 'WP_027648442.1')
('Save', 'WP_028185298.1')
('Save', 'PZM89247.1')
('Save', 'WP_095980633.1')
('Save', 'WP_018221345.1')
('Save', 'GAE86709.1')
('Save', 'WP_025707137.1')
('Save', 'WP_122966474.1')
('Save', 'WP_132396141.1')
('Save', 'WP_154970308.1')
('Save', 'WP_156218292.1')
('Save', 'WP_104303383.1')
('Save', 'WP_045085970.1')
('Save', 'WP_019912658.1')
('Save', 'WP_012166727.1')
('Save', 'WP_080716149.1')
('Save', 'OEU80474.1')
('Save', 'WP_051352696.1')
('Save', 'RMH16883.1')
('Save', 'WP_026736224.1')
('Save', 'PYP93107.1')
('Save', 'WP_066949623.1')
('Save', 'OJY15894.1')
('Save', 'WP_063231157.1')
('Save', 'PZN93824.1')
('Save', 'RYE58907.1')
('Save', 'WP_102170635.1')
('Save', 'WP_153292222.1')
('Save', 'WP_014314299.1')
('Save', 'WP_0

We have extracted all the genbank files from the different proteins identified in the blastp search we will use the genbank files to extract the required ID GCF assemblies that we are going to download using bash scripts.

## 3. Table of GCF-ids

Get the overview table where organism names are linked with their GCF id. This is needed to extract the correct genomic fragment to use as an input in antiSmash.

In [ ]:
#Read file generated from running line from downloadingGenomes.sh
fromGCF = pd.read_csv('referenceGenome.txt', sep = '\t', header = None)
#fromGCF.columns = ["GCF_id", 'Bioproject', 'Organism']
fromGCF.columns = ["GCF_id", 'Organism']
fromGCF.head(10)

Merge both documents

In [ ]:
#Merge
GCFidsToUse = pd.merge(fromOutput, fromGCF, on = 'Organism', how = 'outer')
GCFidsToUse = GCFidsToUse.loc[:,['Organism', 'proteinID', 'GCF_id']]
GCFidsToUse.to_csv('GCFids_organism_protein.txt', sep = '\t')